In [1]:
# grab directory root
import sys
sys.path.append("../")

In [2]:
from dinov3.eval.tSNE import extract_embeddings
from dinov3.data.datasets import NCells
from dinov3.models.backbone_loader import load_backbone
from dinov3.eval.simpleKNN import evaluate_simple_knn
import torch
import json
from torchvision import transforms


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

/home/students/.conda/envs/helmholtz_hannes/lib/python3.10/site-packages/torch/distributed/distributed_c10d.py:4807: UserWarning: No device id is provided via `init_process_group` or `barrier `. Using the current device set by the user. 
  warnings.warn(  # warn only once
[rank0]:[W828 15:37:05.578249094 ProcessGroupNCCL.cpp:5023] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 as device used by this process is currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. You can specify device_id in init_process_group() to force use of a particular device.


In [3]:
# defaul transform used in dinov3
def make_transform(resize_size: int | list[int] = 768):
    to_tensor = transforms.ToTensor()
    resize = transforms.Resize((resize_size, resize_size), antialias=True)
    normalize = transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
    )
    return transforms.Compose([to_tensor, resize, normalize])

test_dataset = NCells(
    root="/home/students/code/helmholtzSS25/Hannes/dinov3playground/manifest_test_fixed.csv.gz",
    split=NCells.Split.TRAIN,
    transform=make_transform(), 
    target_transform=None,
)
print(f"Test Dataset contains {len(test_dataset)} entries")

Test Dataset contains 359426 entries


## Baseline kNN Evaluation

In [9]:
model = load_backbone(
    config_file="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-e80/config.yaml",
    pretrained_weights="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-e80/ckpt/123749",
    output_dir="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-e80"
)
target_size = 224
batch_size = 64
num_workers = 6
embeddings, labels = extract_embeddings(
    model=model,
    dataset=test_dataset,
    device=DEVICE,
    batch_size=batch_size,
    num_workers=num_workers,
    target_size=target_size,
)

Materializing model parameters on cuda
Model loaded on 123750 start iteration
Model backbone architecture: 
 DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (rope_embed): RopePositionEmbedding()
  (blocks): ModuleList(
    (0-11): 12 x SelfAttentionBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): SelfAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        

batches: 100%|██████████| 5617/5617 [12:51<00:00,  7.28it/s]


In [11]:
results = evaluate_simple_knn(embeddings, labels, k_list=[1,5,10], metric='cosine', sample_size=None)
print("kNN Performance:", json.dumps(results, sort_keys=True, indent=4))

kNN Performance: {
    "1": {
        "mAP": 0.522654338451847,
        "precision@k": 0.4745816941456656,
        "recall@k": 0.0002209196774884609,
        "topk_acc": 0.4745816941456656
    },
    "5": {
        "mAP": 0.522654338451847,
        "precision@k": 0.45836862107916504,
        "recall@k": 0.0009804500411107458,
        "topk_acc": 0.6517335974581694
    },
    "10": {
        "mAP": 0.522654338451847,
        "precision@k": 0.45024455659857654,
        "recall@k": 0.001833994614048822,
        "topk_acc": 0.7318891788574005
    }
}


## DBScan (Single Cluster) Finetune Eval

In [4]:
model = load_backbone(
    config_file="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-finetune-dbscan/config.yaml",
    pretrained_weights="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-finetune-dbscan/ckpt/22499",
    output_dir="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-finetune-dbscan"
)
target_size = 224
batch_size = 64
num_workers = 6
embeddings, labels = extract_embeddings(
    model=model,
    dataset=test_dataset,
    device=DEVICE,
    batch_size=batch_size,
    num_workers=num_workers,
    target_size=target_size,
)

Materializing model parameters on cuda
Model loaded on 22500 start iteration
Model backbone architecture: 
 DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (rope_embed): RopePositionEmbedding()
  (blocks): ModuleList(
    (0-11): 12 x SelfAttentionBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): SelfAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (

batches: 100%|██████████| 5617/5617 [09:46<00:00,  9.58it/s]


In [5]:
results = evaluate_simple_knn(embeddings, labels, k_list=[1,5,10], metric='cosine', sample_size=None)
print("kNN Performance:", json.dumps(results, sort_keys=True, indent=4))

kNN Performance: {
    "1": {
        "mAP": 0.5959169510237845,
        "precision@k": 0.5472559024667109,
        "recall@k": 0.000223021184014311,
        "topk_acc": 0.5472559024667109
    },
    "5": {
        "mAP": 0.5959169510237845,
        "precision@k": 0.5332691569335551,
        "recall@k": 0.0010033149822359709,
        "topk_acc": 0.723314395730971
    },
    "10": {
        "mAP": 0.5959169510237845,
        "precision@k": 0.5245956052149815,
        "recall@k": 0.0018819778263280305,
        "topk_acc": 0.8022958828799253
    }
}


## HDBScan Finetune Eval

In [6]:
model = load_backbone(
    config_file="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-finetune-hdbscan/config.yaml",
    pretrained_weights="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-finetune-hdbscan/ckpt/22499",
    output_dir="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-finetune-hdbscan"
)
target_size = 224
batch_size = 64
num_workers = 6
embeddings, labels = extract_embeddings(
    model=model,
    dataset=test_dataset,
    device=DEVICE,
    batch_size=batch_size,
    num_workers=num_workers,
    target_size=target_size,
)

Materializing model parameters on cuda
Model loaded on 22500 start iteration
Model backbone architecture: 
 DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (rope_embed): RopePositionEmbedding()
  (blocks): ModuleList(
    (0-11): 12 x SelfAttentionBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): SelfAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (

batches: 100%|██████████| 5617/5617 [06:19<00:00, 14.78it/s]


In [7]:
results = evaluate_simple_knn(embeddings, labels, k_list=[1,5,10], metric='cosine', sample_size=None)
print("kNN Performance:", json.dumps(results, sort_keys=True, indent=4))

kNN Performance: {
    "1": {
        "mAP": 0.5820977197181092,
        "precision@k": 0.5480738733424961,
        "recall@k": 0.0002242270425645246,
        "topk_acc": 0.5480738733424961
    },
    "5": {
        "mAP": 0.5820977197181092,
        "precision@k": 0.47641906818093294,
        "recall@k": 0.0010104705236814104,
        "topk_acc": 0.7893947571961961
    },
    "10": {
        "mAP": 0.5820977197181092,
        "precision@k": 0.4962565312470439,
        "recall@k": 0.0018909567039771136,
        "topk_acc": 0.8677891972200119
    }
}


## HDBScan Eval (with min_smaple=2)

In [4]:
model = load_backbone(
    config_file="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-finetune-hdbscan2/config.yaml",
    pretrained_weights="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-finetune-hdbscan2/ckpt/22499",
    output_dir="/home/students/code/helmholtzSS25/Hannes/dinov3playground/outputs-finetune-hdbscan2"
)
target_size = 224
batch_size = 64
num_workers = 6
embeddings, labels = extract_embeddings(
    model=model,
    dataset=test_dataset,
    device=DEVICE,
    batch_size=batch_size,
    num_workers=num_workers,
    target_size=target_size,
)

Materializing model parameters on cuda
Model loaded on 22500 start iteration
Model backbone architecture: 
 DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (rope_embed): RopePositionEmbedding()
  (blocks): ModuleList(
    (0-11): 12 x SelfAttentionBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): SelfAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (

batches: 100%|██████████| 5617/5617 [06:27<00:00, 14.50it/s]


In [5]:
results = evaluate_simple_knn(embeddings, labels, k_list=[1,5,10], metric='cosine', sample_size=None)
print("kNN Performance:", json.dumps(results, sort_keys=True, indent=4))

kNN Performance: {
    "1": {
        "mAP": 0.5937017718030762,
        "precision@k": 0.5456199607151403,
        "recall@k": 0.00022136790386864037,
        "topk_acc": 0.5456199607151403
    },
    "5": {
        "mAP": 0.5937017718030762,
        "precision@k": 0.5307351165469387,
        "recall@k": 0.0009926657076118983,
        "topk_acc": 0.719992432378292
    },
    "10": {
        "mAP": 0.5937017718030762,
        "precision@k": 0.5220740847907497,
        "recall@k": 0.001857161567762058,
        "topk_acc": 0.8003483331756746
    }
}
